In [3]:
# v2 imports temperature records for a date range across 1 year using the datetime library
# v3 drops the device_id column
# v4 Changed timedelta(days=1) to timedelta(hours=1) for finer granularity using the datetime library

import requests
import pandas as pd
from datetime import datetime, timedelta

# Step 1: Define the API endpoint
url = "https://api.data.gov.sg/v1/environment/air-temperature"

# Define the start and end dates for the range
start_date = datetime(2023, 11, 1, 0, 0)  # start_date = datetime(yyyy, mm, dd, hh, mm)
end_date = datetime(2024, 10, 31, 23, 0)  # end_date = datetime(yyyy, mm, dd, hh, mm)

# List to hold all the data
all_data = []

# Loop through the date range and request data for each day
current_date = start_date
while current_date <= end_date:
    date_str = current_date.strftime("%Y-%m-%dT%H:%M:%S")  # Format date for the API request
    params = {"date_time": date_str}  # Use the 'date_time' parameter for the API request
    
    # Send the GET request to the API
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        json_data = response.json()
        
        # Extract and flatten the "readings" for each station
        items = json_data["items"]
        readings = [reading for item in items for reading in item["readings"]]
        
        # Create a DataFrame from the readings
        df = pd.DataFrame(readings)
        
        # Merge with station metadata
        stations = pd.DataFrame(json_data["metadata"]["stations"])
        df = df.merge(stations, left_on="station_id", right_on="id", how="left")
        
        # Drop the 'id' column to clean up the DataFrame, and drop 'device_id' column
        df.drop(columns=["id"], inplace=True)
        df.drop(columns=["device_id"], inplace=True)
        
        # Add the current date to the DataFrame for easy tracking
        df["date"] = date_str
        
        # Append the data for this day to the all_data list
        all_data.append(df)
    else:
        print(f"Failed to fetch data for {date_str} with status code {response.status_code}")
    
    # Increment the current date by one hour
    current_date += timedelta(hours=1)

# If any data was fetched, concatenate all data frames into one
if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    
    # Convert 'date' column to datetime format
    final_df["date"] = pd.to_datetime(final_df["date"])

    # Preview the final DataFrame
    print(final_df.head())
    print(final_df.info())
    display(final_df)
    
    # Optional: Save the data to a CSV file
    final_df.to_csv('airtempacrosssg_hourly.csv', index=False)
else:
    print("No data was fetched during the specified date range.")

  station_id  value                 name  \
0       S109   26.6  Ang Mo Kio Avenue 5   
1       S117   27.4          Banyan Road   
2        S50   26.4        Clementi Road   
3       S107   28.8   East Coast Parkway   
4        S43   28.1       Kim Chuan Road   

                                      location       date  
0  {'latitude': 1.3764, 'longitude': 103.8492} 2023-11-01  
1    {'latitude': 1.256, 'longitude': 103.679} 2023-11-01  
2  {'latitude': 1.3337, 'longitude': 103.7768} 2023-11-01  
3  {'latitude': 1.3135, 'longitude': 103.9625} 2023-11-01  
4  {'latitude': 1.3399, 'longitude': 103.8878} 2023-11-01  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114095 entries, 0 to 114094
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   station_id  114095 non-null  object        
 1   value       114095 non-null  float64       
 2   name        114095 non-null  object        
 3   location    114

,station_id,value,name,location,date
0,S109,26.6,Ang Mo Kio Avenue 5,"{'latitude': 1.3764, 'longitude': 103.8492}",2023-11-01 00:00:00
1,S117,27.4,Banyan Road,"{'latitude': 1.256, 'longitude': 103.679}",2023-11-01 00:00:00
2,S50,26.4,Clementi Road,"{'latitude': 1.3337, 'longitude': 103.7768}",2023-11-01 00:00:00
3,S107,28.8,East Coast Parkway,"{'latitude': 1.3135, 'longitude': 103.9625}",2023-11-01 00:00:00
4,S43,28.1,Kim Chuan Road,"{'latitude': 1.3399, 'longitude': 103.8878}",2023-11-01 00:00:00
...,...,...,...,...,...
114090,S60,29.5,Sentosa,"{'latitude': 1.25, 'longitude': 103.8279}",2024-10-31 23:00:00
114091,S115,29.6,Tuas South Avenue 3,"{'latitude': 1.29377, 'longitude': 103.61843}",2024-10-31 23:00:00
114092,S24,28.0,Upper Changi Road North,"{'latitude': 1.3678, 'longitude': 103.9826}",2024-10-31 23:00:00
114093,S116,29.6,West Coast Highway,"{'latitude': 1.281, 'longitude': 103.754}",2024-10-31 23:00:00


In [ ]:
from sqlalchemy import create_engine

# PostgreSQL connection details
host = "localhost"
port = "5432"
dbname = "your_database"
user = "your_username"
password = "your_password"

# Create a connection string
connection_string = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}"

# Create a SQLAlchemy engine
engine = create_engine(connection_string)

# Load the DataFrame into a PostgreSQL table
table_name = "your_table"  # Replace with your table name
df.to_sql(table_name, engine, if_exists="replace", index=False)  # Use 'append' instead of 'replace' to add data
